In [1]:
#importing required libraries
import os
import shutil
import time

import folium
import imageio
import webbrowser
import zipfile
import json
import fileinput

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os import path
from branca.colormap import linear
from selenium import webdriver
from PIL import Image
from pathlib import Path

from glob import glob
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
#reading 2020 data into respective dataframes
df1 = pd.read_csv('../Data/2020/Divvy_Trips_2020_Q1.csv')
df2 = pd.read_csv('../Data/2020/202004-divvy-tripdata.csv')
df3 = pd.read_csv('../Data/2020/202005-divvy-tripdata.csv')
df4 = pd.read_csv('../Data/2020/202006-divvy-tripdata.csv')
df5 = pd.read_csv('../Data/2020/202007-divvy-tripdata.csv')
df6 = pd.read_csv('../Data/2020/202008-divvy-tripdata.csv')
df7 = pd.read_csv('../Data/2020/202009-divvy-tripdata.csv')
df8 = pd.read_csv('../Data/2020/202010-divvy-tripdata.csv')
df9 = pd.read_csv('../Data/2020/202011-divvy-tripdata.csv')
df10 = pd.read_csv('../Data/2020/202012-divvy-tripdata.csv')

In [3]:
#Concatenating the dataframes into a single dataframe
frames = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10]
dfbike = pd.concat(frames)
dfbike.reset_index(drop=True,inplace=True)
dfbike.head(5)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,EACB19130B0CDA4A,docked_bike,2020-01-21 20:06:59,2020-01-21 20:14:30,Western Ave & Leland Ave,239,Clark St & Leland Ave,326,41.9665,-87.6884,41.9671,-87.6674,member
1,8FED874C809DC021,docked_bike,2020-01-30 14:22:39,2020-01-30 14:26:22,Clark St & Montrose Ave,234,Southport Ave & Irving Park Rd,318,41.9616,-87.6660,41.9542,-87.6644,member
2,789F3C21E472CA96,docked_bike,2020-01-09 19:29:26,2020-01-09 19:32:17,Broadway & Belmont Ave,296,Wilton Ave & Belmont Ave,117,41.9401,-87.6455,41.9402,-87.6530,member
3,C9A388DAC6ABF313,docked_bike,2020-01-06 16:17:07,2020-01-06 16:25:56,Clark St & Randolph St,51,Fairbanks Ct & Grand Ave,24,41.8846,-87.6319,41.8918,-87.6206,member
4,943BC3CBECCFD662,docked_bike,2020-01-30 08:37:16,2020-01-30 08:42:48,Clinton St & Lake St,66,Wells St & Hubbard St,212,41.8856,-87.6418,41.8899,-87.6343,member


In [4]:
dfbike.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

### Saving all the station names in a list "allSt20"

#### Comparing with 2019 data and only adding the stations which are new into a list "added20"
#### Also, checking for the stations that have been in 2019 but not in this year's data and adding it to "removed19"

In [5]:
allSt20 = list(dfbike['start_station_name'].unique())
len(allSt20)

695

In [6]:
allSt19 = ['Wabash Ave & Grand Ave',
 'State St & Randolph St',
 'Racine Ave & 18th St',
 'California Ave & Milwaukee Ave',
 'Mies van der Rohe Way & Chicago Ave',
 'LaSalle St & Washington St',
 'St. Clair St & Erie St',
 'Fort Dearborn Dr & 31st St',
 'Lake Shore Dr & North Blvd',
 'Halsted St & Roscoe St',
 'Prairie Ave & Garfield Blvd',
 'Millennium Park',
 'Wells St & Concord Ln',
 'Lincoln Ave & Diversey Pkwy',
 'Streeter Dr & Grand Ave',
 'State St & Kinzie St',
 'Michigan Ave & Oak St',
 'Wilton Ave & Diversey Pkwy',
 'Greenview Ave & Jarvis Ave',
 'Broadway & Sheridan Rd',
 'Pine Grove Ave & Irving Park Rd',
 'Clinton St & Lake St',
 'Damen Ave & Foster Ave',
 'Ellis Ave & 53rd St',
 'Larrabee St & Webster Ave',
 'Winthrop Ave & Lawrence Ave',
 'Broadway & Waveland Ave',
 'Halsted St & Archer Ave',
 'Bosworth Ave & Howard St',
 'State St & 29th St',
 'Kedzie Ave & Milwaukee Ave',
 'Orleans St & Hubbard St (*)',
 'Sheridan Rd & Irving Park Rd',
 'Lakeview Ave & Fullerton Pkwy',
 'Kedzie Ave & Palmer Ct',
 'Clark St & Bryn Mawr Ave',
 'Sedgwick St & Schiller St',
 'Wabash Ave & Wacker Pl',
 'Racine Ave & 35th St',
 'Damen Ave & Sunnyside Ave',
 'Larrabee St & Division St',
 'Morgan St & Lake St',
 'Noble St & Milwaukee Ave',
 'Lincoln Ave & Roscoe St',
 'Clinton St & Madison St',
 'Clark St & Armitage Ave',
 'Bissell St & Armitage Ave',
 'Michigan Ave & 14th St',
 'Eckhart Park',
 'Damen Ave & Chicago Ave',
 'Racine Ave & Fullerton Ave',
 'Sheffield Ave & Waveland Ave',
 'Western Ave & Leland Ave',
 'Wilton Ave & Belmont Ave',
 'Southport Ave & Irving Park Rd',
 'Wells St & Evergreen Ave',
 'Michigan Ave & Madison St',
 'Halsted St & Clybourn Ave (*)',
 'Wells St & Walton St',
 'Sheffield Ave & Wellington Ave',
 'Paulina Ave & North Ave',
 'State St & Pearson St',
 'Sheridan Rd & Montrose Ave',
 'Racine Ave & Belmont Ave',
 'Southport Ave & Roscoe St',
 'Wabash Ave & 16th St',
 'Fairbanks Ct & Grand Ave',
 'Damen Ave & Pierce Ave',
 'Michigan Ave & 18th St',
 'Racine Ave (May St) & Fulton St',
 'Sheridan Rd & Lawrence Ave',
 'Clark St & Drummond Pl',
 'Campbell Ave & Montrose Ave',
 'Mies van der Rohe Way & Chestnut St',
 'Campbell Ave & Fullerton Ave',
 'Dearborn St & Monroe St',
 'Rush St & Superior St',
 'Kimball Ave & Belmont Ave',
 'Kedzie Ave & Leland Ave',
 'Wabash Ave & 9th St',
 'Troy St & North Ave',
 'Green St & Madison St',
 'Halsted St & Willow St',
 'Wabash Ave & Roosevelt Rd',
 'Burling St (Halsted) & Diversey Pkwy (Temp)',
 'Kingsbury St & Kinzie St',
 'Orleans St & Merchandise Mart Plaza',
 'Wolcott (Ravenswood) Ave & Montrose Ave (*)',
 'Lake Park Ave & 53rd St',
 'LaSalle Dr & Huron St (*)',
 'Sheridan Rd & Buena Ave',
 'Clark St & Newport St',
 'Ashland Ave & Division St',
 'Western Ave & Winnebago Ave',
 'Daley Center Plaza',
 'Sheffield Ave & Fullerton Ave',
 'LaSalle St & Illinois St',
 'Ashland Ave & Chicago Ave',
 'Ridge Blvd & Howard St',
 'Spaulding Ave & Armitage Ave',
 'Avondale Ave & Irving Park Rd',
 'Wentworth Ave & 35th St',
 'State St & 76th St',
 'Francisco Ave & Foster Ave',
 'Damen Ave & Clybourn Ave',
 'Field Blvd & South Water St',
 'Lincoln Ave & Waveland Ave',
 'Michigan Ave & Lake St',
 'Clark St & 9th St (AMLI)',
 'Peoria St & Jackson Blvd',
 'Kosciuszko Park',
 'Central Park Blvd & 5th Ave',
 'Desplaines St & Kinzie St',
 'Emerald Ave & 31st St',
 'Sedgwick St & Huron St',
 'Canal St & Madison St',
 'Michigan Ave & Washington St',
 'Clinton St & Roosevelt Rd',
 'Lake Shore Dr & Monroe St',
 'Loomis St & Lexington St',
 'Loomis St & Taylor St (*)',
 'Ashland Ave & 13th St',
 'Franklin St & Chicago Ave',
 'Paulina St & Flournoy St',
 'Dusable Harbor',
 'Clifton Ave & Armitage Ave',
 'Adler Planetarium',
 'Shore Dr & 55th St',
 'Leavitt St & Chicago Ave',
 'Aberdeen St & Jackson Blvd',
 'Broadway & Cornelia Ave',
 'Ravenswood Ave & Lawrence Ave',
 'Ritchie Ct & Banks St',
 'Lincoln Ave & Sunnyside Ave',
 'Orleans St & Chestnut St (NEXT Apts)',
 'Lake Shore Dr & Belmont Ave',
 'Clark St & Lake St',
 'Theater on the Lake',
 'Wood St & Milwaukee Ave',
 'Dearborn Pkwy & Delaware Pl',
 'Halsted St & 35th St (*)',
 'Clark St & Wrightwood Ave',
 'Manor Ave & Leland Ave',
 'Lake Shore Dr & Ohio St',
 'Sedgwick St & North Ave',
 'Dearborn St & Erie St',
 'Cityfront Plaza Dr & Pioneer Ct',
 'Wabash Ave & Cermak Rd',
 'Stockton Dr & Wrightwood Ave',
 'Broadway & Granville Ave',
 'Rush St & Hubbard St',
 'Wolcott Ave & Polk St',
 'Western Ave & Roscoe St',
 'Sedgwick St & Webster Ave',
 'Larrabee St & Oak St',
 'Columbus Dr & Randolph St',
 'Clarendon Ave & Gordon Ter',
 'Pine Grove Ave & Waveland Ave',
 'Western Ave & Lunt Ave',
 'Halsted St & 37th St',
 'Racine Ave & Congress Pkwy',
 'Milwaukee Ave & Grand Ave',
 'Clark St & Berwyn Ave',
 'Wells St & Elm St',
 'Aberdeen St & Monroe St',
 'Normal Ave & Archer Ave',
 'Clark St & Schiller St',
 'Clinton St & Tilden St',
 'Racine Ave & Randolph St',
 'Broadway & Belmont Ave',
 'Leavitt St & Division St (*)',
 'Rush St & Cedar St',
 'Paulina St & 18th St',
 'Canal St & Adams St',
 'Sheffield Ave & Webster Ave',
 'Lake Shore Dr & Wellington Ave',
 'Jefferson St & Monroe St',
 'Southport Ave & Wrightwood Ave',
 'Indiana Ave & Roosevelt Rd',
 'Ashland Ave & Wellington Ave',
 'Clarendon Ave & Leland Ave',
 'Vernon Ave & 75th St',
 'Clark St & Ida B Wells Dr',
 'Clark St & Randolph St',
 'Western Ave & Walton St',
 'McClurg Ct & Illinois St',
 'Clark St & Grace St',
 'Ada St & Washington Blvd',
 'Leavitt St & North Ave',
 'Campbell Ave & North Ave',
 'Ogden Ave & Chicago Ave',
 'State St & 19th St',
 'Clark St & Lincoln Ave',
 'Ashland Ave & Grand Ave',
 'Greenview Ave & Fullerton Ave',
 'Federal St & Polk St',
 'Clark St & Wellington Ave',
 'Lincoln Ave & Fullerton Ave',
 'Damen Ave & Division St',
 'Clybourn Ave & Division St',
 'Clark St & Elm St',
 'Clark St & Chicago Ave',
 'Southport Ave & Clark St',
 'Desplaines St & Jackson Blvd',
 'Ravenswood Ave & Irving Park Rd',
 'Broadway & Argyle St',
 'Wells St & Huron St',
 'Indiana Ave & 31st St',
 'Central St Metra',
 'Clark St & Schreiber Ave',
 'Halsted St & Wrightwood Ave',
 'Financial Pl & Ida B Wells Dr (Temp)',
 'Southport Ave & Belmont Ave',
 'Sheridan Rd & Greenleaf Ave',
 'McClurg Ct & Erie St',
 'Winchester Ave & Elston Ave',
 'Larrabee St & North Ave',
 'Blackstone Ave & Hyde Park Blvd',
 'Sheffield Ave & Willow St',
 'Damen Ave & Melrose Ave',
 'Broadway & Wilson Ave',
 'Ravenswood Ave & Berteau Ave',
 'Ellis Ave & 58th St',
 'Claremont Ave & Hirsch St',
 'Halsted St & Dickens Ave',
 'Lincoln Ave & Belmont Ave',
 'Franklin St & Jackson Blvd',
 'Michigan Ave & Pearson St',
 'State St & Van Buren St',
 'Honore St & Division St',
 'Orleans St & Elm St (*)',
 'Aberdeen St & Randolph St',
 'Western Ave & 24th St',
 'Canal St & Taylor St',
 'Broadway & Barry Ave',
 'Wells St & Polk St',
 'Wells St & 19th St',
 'Marine Dr & Ainslie St',
 'Clark St & Montrose Ave',
 'California Ave & 23rd Pl',
 'Southport Ave & Wellington Ave',
 'Canal St & Harrison St',
 'Ashland Ave & Archer Ave',
 'Broadway & Berwyn Ave',
 'Wood St & Chicago Ave (*)',
 'Glenwood Ave & Touhy Ave',
 'Ashland Ave & Lake St (Temp)',
 'Franklin St & Lake St',
 'Blue Island Ave & 18th St',
 'Wentworth Ave & 33rd St',
 'Lincoln Ave & Belle Plaine Ave',
 'Lakefront Trail & Bryn Mawr Ave',
 'MLK Jr Dr & Pershing Rd',
 'Canal St & Monroe St (*)',
 'Clark St & North Ave',
 'Desplaines St & Randolph St',
 'LaSalle St & Jackson Blvd',
 'Loomis St & Jackson Blvd',
 'Kedzie Ave & Bryn Mawr Ave',
 'Clinton St & Jackson Blvd (*)',
 'Lakefront Trail & Wilson Ave',
 'Larrabee St & Kingsbury St',
 'Wells St & Hubbard St',
 'Sheffield Ave & Wrightwood Ave',
 'Burnham Harbor',
 'Sheffield Ave & Kingsbury St',
 'Stave St & Armitage Ave',
 'Kingsbury St & Erie St',
 'Ashland Ave & Wrightwood Ave',
 'Morgan St & 31st St',
 'Lake Shore Dr & Diversey Pkwy',
 'Larrabee St & Menomonee St',
 'Kedzie Ave & Lake St',
 'Damen Ave & Leland Ave',
 'May St & Cullerton St',
 'Artesian Ave & Hubbard St',
 'Western Ave & Monroe St',
 'Halsted St & North Branch St',
 'Seeley Ave & Roscoe St',
 'Harper Ave & 59th St',
 'Milwaukee Ave & Wabansia Ave',
 'Damen Ave & Thomas St (Augusta Blvd)',
 'May St & Taylor St',
 'Eastlake Ter & Rogers Ave',
 'California Ave & Montrose Ave',
 'Damen Ave & Cortland St',
 'California Ave & Altgeld St',
 'California Ave & Francis Pl',
 'Benson Ave & Church St',
 'California Ave & North Ave',
 'Michigan Ave & Jackson Blvd',
 'Leavitt St & Archer Ave',
 'Michigan Ave & Ida B Wells Dr',
 'Milwaukee Ave & Rockwell St',
 'Green St & Randolph St',
 'Damen Ave & Grand Ave',
 'Sheridan Rd & Noyes St (NU)',
 'Clinton St & Washington Blvd',
 'MLK Jr Dr & 29th St',
 'Kimbark Ave & 53rd St',
 'Glenwood Ave & Morse Ave',
 'State St & Harrison St',
 'Wentworth Ave & Cermak Rd (Temp)',
 'Calumet Ave & 18th St',
 'Clark St & Winnemac Ave',
 'Marshfield Ave & Cortland St',
 'Dearborn St & Van Buren St (*)',
 'Racine Ave & Wrightwood Ave',
 'Greenview Ave & Diversey Pkwy',
 'Albany Ave & Bloomingdale Ave',
 'MLK Jr Dr & 63rd St',
 'Ashland Ave & Grace St',
 'Oakley Ave & Irving Park Rd',
 'Troy St & Elston Ave',
 'Dearborn St & Adams St',
 'Damen Ave & Wellington Ave',
 'Richmond St & Diversey Ave',
 'Broadway & Ridge Ave',
 'Field Museum',
 'Morgan St & Polk St',
 'Logan Blvd & Elston Ave',
 'Wood St & Taylor St',
 'Cottage Grove Ave & 63rd St',
 'Ogden Ave & Race Ave',
 'Southport Ave & Waveland Ave',
 'Calumet Ave & 33rd St',
 'University Ave & 57th St',
 'Delano Ct & Roosevelt Rd',
 'Lake Park Ave & 47th St',
 '63rd St Beach',
 'Hermitage Ave & Polk St',
 'Franklin St & Monroe St',
 'Dayton St & North Ave',
 'Hoyne Ave & Balmoral Ave',
 'Dodge Ave & Mulford St',
 'Ashland Ave & 21st St',
 'Dorchester Ave & 49th St',
 'Sheridan Rd & Loyola Ave',
 'Lincoln Ave & Winona St',
 'Wabash Ave & Adams St',
 'Sheridan Rd & Columbia Ave',
 'Buckingham Fountain (Temp)',
 'Western Ave & Division St',
 'Knox Ave & Montrose Ave',
 'Emerald Ave & 28th St',
 'Wentworth Ave & 63rd St',
 'Sawyer Ave & Irving Park Rd',
 'Ellis Ave & 60th St',
 'Sacramento Blvd & Franklin Blvd',
 'Damen Ave & Madison St',
 'LaSalle St & Adams St',
 'Drake Ave & Addison St',
 'Ashland Ave & Blackhawk St',
 'Clarendon Ave & Junior Ter',
 'Rockwell St & Eastwood Ave',
 'Canal St & Jackson Blvd',
 'Kedzie Ave & 24th St',
 'Clark St & Elmdale Ave',
 '900 W Harrison St',
 'Fairbanks St & Superior St (*)',
 'Franklin St & Quincy St',
 'Wood St & Augusta Blvd',
 'Loomis St & Archer Ave',
 'Wacker Dr & Washington St',
 'McCormick Place',
 'Ashland Ave & Augusta Blvd',
 'Leavitt St & Addison St',
 'Western Ave & 21st St',
 'Central Park Ave & North Ave',
 'Lincolnwood Dr & Central St',
 'Ashland Ave & Belle Plaine Ave',
 'Humboldt Blvd & Armitage Ave',
 'Kedzie Ave & Foster Ave',
 'Damen Ave & Charleston St',
 'Morgan Ave & 14th Pl',
 'California Ave & Cortez St',
 'Warren Park East',
 'Carpenter St & Huron St',
 'Leavitt St & Lawrence Ave',
 'Halsted St & Polk St',
 'Sangamon St & Washington Blvd (*)',
 'Paulina St & Montrose Ave',
 'Southport Ave & Clybourn Ave',
 'Stetson Ave & South Water St',
 'Woodlawn Ave & 55th St',
 'Western Ave & Congress Pkwy',
 'Shields Ave & 31st St',
 'Broadway & Thorndale Ave',
 'Drake Ave & Fullerton Ave',
 'Pulaski Rd & Eddy St (Temp)',
 'Clark St & Leland Ave',
 'Wood St & Hubbard St',
 'Clark St & Columbia Ave',
 'Monticello Ave & Irving Park Rd',
 'Drake Ave & Montrose Ave',
 'Kilbourn Ave & Milwaukee Ave',
 'Ellis Ave & 55th St',
 'Ogden Ave & Congress Pkwy',
 'Washtenaw Ave & Lawrence Ave',
 'Halsted St & 21st St',
 'Shedd Aquarium',
 'Calumet Ave & 21st St',
 'MLK Jr Dr & 47th St',
 'Lincoln Ave & Addison St',
 'Winchester (Ravenswood) Ave & Balmoral Ave',
 'Lake Park Ave & 56th St',
 'Racine Ave & 15th St',
 'Central Park Ave & Elbridge Ave',
 'Central Park Ave & Bloomingdale Ave',
 'Larrabee St & Armitage Ave',
 'Racine Ave & Washington Blvd (*)',
 'Morgan St & 18th St',
 'Elston Ave & Wabansia Ave',
 'Woodlawn Ave & Lake Park Ave',
 'Halsted St & 18th St',
 'Wood St & 35th St',
 'Evanston Civic Center',
 'California Ave & Byron St',
 'Halsted St & 47th Pl',
 'Kilbourn Ave & Irving Park Rd',
 'Michigan Ave & 8th St',
 'Malcolm X College',
 'Racine Ave & 13th St',
 'Talman Ave & Addison St',
 'Ogden Ave & Roosevelt Rd',
 'Clinton St & 18th St',
 'California Ave & Fletcher St',
 'California Ave & Division St',
 'Calumet Ave & 35th St',
 'Elmwood Ave & Austin St',
 'Wallace St & 35th St',
 'Museum of Science and Industry',
 'Kedzie Ave & 21st St',
 'Cottage Grove Ave & 43rd St',
 'Kildare Ave & Montrose Ave',
 'State St & 33rd St',
 'Washtenaw Ave & Ogden Ave (*)',
 'Chicago Ave & Dempster St',
 'Cornell Ave & Hyde Park Blvd',
 'California Ave & Lake St',
 'Dodge Ave & Church St',
 'Eggleston Ave & 69th St (*)',
 'Spaulding Ave & Division St',
 'Cannon Dr & Fullerton Ave',
 'Damen Ave & Coulter St',
 'Leavitt St & Armitage Ave',
 'Shields Ave & 28th Pl',
 'State St & 35th St',
 'Halsted St & Roosevelt Rd',
 'Clinton St & Polk St (*)',
 'Elizabeth St & 47th St',
 'University Library (NU)',
 'Walsh Park',
 'Bernard St & Elston Ave',
 'Clark St & Lunt Ave',
 'Wentworth Ave & 24th St',
 'Halsted St & Maxwell St',
 'Valli Produce - Evanston Plaza',
 'Chicago Ave & Washington St',
 'Western Ave & Howard St',
 'Rhodes Ave & 32nd St',
 'Montrose Harbor',
 'Western Ave & 28th St',
 'Christiana Ave & Lawrence Ave',
 'Budlong Woods Library',
 'Damen Ave & Cullerton St',
 'Perry Ave & 69th St',
 'Greenwood Ave & 47th St',
 'Maplewood Ave & Peterson Ave',
 'Indiana Ave & 26th St',
 'Princeton Ave & Garfield Blvd',
 'Cottage Grove Ave & Oakwood Blvd',
 'South Shore Dr & 71st St',
 '2112 W Peterson Ave',
 'MLK Jr Dr & 56th St (*)',
 'Jeffery Blvd & 67th St',
 'Milwaukee Ave & Cuyler Ave',
 'Laramie Ave & Madison St',
 'Keystone Ave & Montrose Ave',
 'Kedzie Ave & Chicago Ave',
 'Ashland Ave & McDowell Ave',
 'Hoyne Ave & 47th St',
 'Central St & Girard Ave',
 'South Shore Dr & 67th St',
 'Kedzie Ave & Roosevelt Rd',
 'Paulina St & Howard St',
 'Cottage Grove Ave & 47th St',
 'Western Ave & Fillmore St (*)',
 'Stony Island Ave & 67th St',
 'Damen Ave & Walnut (Lake) St (*)',
 'Clark St & Jarvis Ave',
 'Smith Park (*)',
 'Warren Park West',
 'Princeton Ave & 47th St',
 'Western Ave & Granville Ave',
 'Leavitt St & Belmont Ave (*)',
 'Avers Ave & Belmont Ave',
 'Ridge Blvd & Touhy Ave',
 'California Ave & 21st St',
 'St. Louis Ave & Balmoral Ave',
 'Lake Park Ave & 35th St',
 'Clark St & Touhy Ave',
 'Chicago Ave & Sheridan Rd',
 'Albany Ave & Montrose Ave',
 'Prairie Ave & 43rd St',
 'Wolcott Ave & Fargo Ave',
 'Rainbow Beach',
 'Cottage Grove Ave & 51st St',
 'DuSable Museum',
 'Cottage Grove Ave & 67th St',
 'Jeffery Blvd & 71st St',
 'Ashland Ave & Pershing Rd',
 'South Shore Dr & 74th St',
 'Conservatory Dr & Lake St',
 'Pulaski Rd & Lake St',
 'Halsted St & 63rd St',
 'Eberhart Ave & 61st St',
 'Oakley Ave & Touhy Ave',
 'California Ave & 26th St',
 'Keystone Ave & Fullerton Ave',
 'Dorchester Ave & 63rd St',
 'Calumet Ave & 51st St',
 'Austin Blvd & Lake St',
 'Kostner Ave & Adams St',
 'Indiana Ave & 40th St',
 'Fairfield Ave & Roosevelt Rd',
 'Central Park Ave & Ogden Ave',
 'Damen Ave & 51st St',
 'Laramie Ave & Gladys Ave',
 'Cicero Ave & Quincy St',
 'Karlov Ave & Madison St',
 'State St & 54th St',
 'Marshfield Ave & 44th St',
 'State St & Pershing Rd',
 'Archer (Damen) Ave & 37th St (*)',
 'Union Ave & Root St',
 'State St & 79th St',
 'Western Blvd & 48th Pl',
 'Wabash Ave & 87th St',
 'Cottage Grove Ave & 83rd St',
 'Pulaski Rd & Congress Pkwy',
 'Kenton Ave & Madison St',
 'Damen Ave & Pershing Rd',
 'Greenwood Ave & 79th St',
 'Throop St & 52nd St',
 'Racine Ave & Garfield Blvd',
 'Ashland Ave & 66th St',
 'Racine Ave & 61st St',
 'Damen Ave & 59th St',
 'Central Ave & Madison St',
 'Evans Ave & 75th St',
 'Bennett Ave & 79th St',
 'Austin Blvd & Chicago Ave',
 'Phillips Ave & 79th St',
 'Cottage Grove Ave & 71st St',
 'Millard Ave & 26th St',
 'Albany Ave & 26th St',
 'Cornell Dr & Hayes Dr',
 'Jeffery Blvd & 76th St',
 'Calumet Ave & 71st St',
 'MLK Jr Dr & 83rd St',
 'Central Park Ave & 24th St',
 'Shields Ave & 43rd St',
 'Cicero Ave & Lake St',
 'Austin Blvd & Madison St',
 'DIVVY Map Frame B/C Station',
 'Yates Blvd & 75th St',
 'Stony Island Ave & 64th St',
 'Stony Island Ave & 75th St',
 'Central Ave & Chicago Ave',
 'Stony Island Ave & 71st St',
 'Kostner Ave & Lake St',
 'Halsted St & 69th St',
 'Ashland Ave & 63rd St',
 'Stony Island Ave & 82nd St',
 'South Chicago Ave & 83rd St',
 'Ashland Ave & 69th St',
 'Stewart Ave & 63rd St (*)',
 'May St & 69th St',
 'Exchange Ave & 79th St',
 'Morgan St & Pershing Rd',
 'Phillips Ave & 83rd St',
 'Elizabeth St & 59th St',
 'Halsted St & 51st St',
 'Carpenter St & 63rd St',
 'Kedzie Ave & Harrison St',
 'Stony Island Ave & South Chicago Ave',
 'South Chicago Ave & Elliot Ave',
 'Ashland Ave & Garfield Blvd',
 'Central Ave & Lake St',
 'Cicero Ave & Flournoy St',
 'Ashland Ave & 50th St',
 'Vernon Ave & 79th St',
 'Wabash Ave & 83rd St',
 'Dearborn St & Van Buren St',
 'Halsted St & Clybourn Ave',
 'LaSalle Dr & Huron St',
 'Fairbanks St & Superior St',
 'Franklin St & Chicago Ave (Temp)',
 'Franklin St & Adams St (Temp)',
 'Canal St & Monroe St',
 'Sangamon St & Washington Blvd',
 'Throop (Loomis) St & Taylor St',
 'Financial Pl & Ida B Wells Dr',
 'Clinton St & Jackson Blvd',
 'Orleans St & Elm St',
 'Ashland Ave & Lake St',
 'Archer (Damen) Ave & 37th St',
 'Orleans St & Hubbard St',
 'Wolcott (Ravenswood) Ave & Montrose Ave',
 'Elizabeth (May) St & Fulton St',
 'Clinton St & Polk St',
 'Halsted St & 35th St',
 'Buckingham Fountain',
 'Wentworth Ave & 24th St (Temp)',
 'Commercial Ave & 83rd St',
 'Washtenaw Ave & Ogden Ave',
 'MLK Jr Dr & 56th St',
 'Central Ave & Harrison St',
 'Michigan Ave & 71st St',
 'Latrobe Ave & Chicago Ave',
 'Halsted St & 59th St',
 'Woodlawn Ave & 75th St',
 'Laramie Ave & Kinzie St',
 'Seeley Ave & Garfield Blvd',
 'Vincennes Ave & 75th St',
 'Cottage Grove Ave & 78th St',
 'Halsted St & 56th St',
 'HUBBARD ST BIKE CHECKING (LBS-WH-TEST)',
 'Marshfield Ave & 59th St',
 'Rhodes Ave & 71st St',
 'Ellis Ave & 83rd St',
 'Cherry Ave & Blackhawk St',
 'Franklin St & Illinois St',
 'Lincoln Park Conservatory',
 'Racine Ave & 65th St',
 'California Ave & Francis Pl (Temp)']

In [7]:
len(allSt19)

637

In [8]:
removed19 = list(set(allSt19) - set(allSt20))
len(removed19)

26

In [9]:
added20 = list(set(allSt20) - set(allSt19))
added20.pop(0)

nan

In [10]:
len(added20)

83

# Plotting all added stations of 2020 and removed stations of 2019

In [11]:
#reading the 'final_stations.csv' into a dataframe
#using all the stations co-ordinates and comparing with each of the year's stations added/ removed list and adding locations 
#     to each of the list to plot yearly data.

final = pd.read_csv('final_stations.csv')

In [12]:
final.head(1)

,Unnamed: 0,Station,Latitude,Longitude
0,0,Ogden Ave & Race Ave,41.891795,-87.658751


In [13]:
#Removing the unnecessary column in the dataframe
final = final.drop('Unnamed: 0', axis =1)

In [14]:
final.head(1)

,Station,Latitude,Longitude
0,Ogden Ave & Race Ave,41.891795,-87.658751


In [15]:
#Converting the dataframe into a dictionary
# key : Station name and value: latitude and longitude

locations = dict()

for station in final['Station']:
    locations[station] = {'lat': final[final['Station'] == station]['Latitude'].to_list()[0], 'long': final[final['Station'] == station]['Longitude'].to_list()[0]}

#printing out the first item in the dictionary to check the key-value pair
list(locations.items())[0]

('Ogden Ave & Race Ave', {'lat': 41.891795, 'long': -87.658751})

In [16]:
#Adding co-ordinates to all the stations that are added this year
n20 = {k: locations[k] for k in locations.keys() & set(added20)}

#printing out the first item in the dictionary to check the key-value pair
list(n20.items())[0]

('Big Marsh Park', {'lat': 41.685877056, 'long': -87.573823929})

In [17]:
#Converting the dictionary into dataframe
st20 = pd.DataFrame.from_dict(n20, orient='index')

In [18]:
#Resetting the index
st20.reset_index(inplace=True)

#Renaming the index column to Station name
st20 = st20.rename(columns = {'index':'Station Name'})

In [19]:
#Adding a column with values of color for every station
st20['color'] = st20.apply(lambda row: 'black', axis=1)

#printing to check the structure of the dataframe
st20.head(1)

,Station Name,lat,long,color
0,Big Marsh Park,41.685877,-87.573824,black


In [20]:
removed19 = ['LaSalle Dr & Huron St (*)',
 'Fairbanks St & Superior St (*)',
 'Orleans St & Elm St (*)',
 'Ashland Ave & Lake St (Temp)',
 'Canal St & Monroe St (*)',
 'Washtenaw Ave & Ogden Ave (*)',
 'Racine Ave (May St) & Fulton St',
 'Halsted St & Clybourn Ave (*)',
 'Financial Pl & Ida B Wells Dr (Temp)',
 'Dearborn St & Van Buren St (*)',
 'Sangamon St & Washington Blvd (*)',
 'Wentworth Ave & 24th St',
 'California Ave & Francis Pl',
 'DIVVY Map Frame B/C Station',
 'Loomis St & Taylor St (*)',
 'Wolcott (Ravenswood) Ave & Montrose Ave (*)',
 'Buckingham Fountain (Temp)',
 'Franklin St & Quincy St',
 'Archer (Damen) Ave & 37th St (*)',
 'Ashland Ave & 21st St',
 'Clinton St & Jackson Blvd (*)',
 'Franklin St & Chicago Ave (Temp)',
 'Orleans St & Hubbard St (*)',
 'Halsted St & 35th St (*)',
 'Clinton St & Polk St (*)',
 'MLK Jr Dr & 56th St (*)']

In [21]:
#adding co-ordinates for the removed stations in 2019
rm19 = {k: locations[k] for k in locations.keys() & set(removed19)}

#printing out the first item in the dictionary to check the key-value pair
list(rm19.items())[0]

('Orleans St & Elm St (*)', {'lat': 41.902924, 'long': -87.637715})

In [22]:
#Converting the dictionary into dataframe
rm19 = pd.DataFrame.from_dict(rm19, orient='index')

In [23]:
#Resetting the index
rm19.reset_index(inplace=True)

#Renaming the index column to Station name
rm19 = rm19.rename(columns = {'index':'Station Name'})

In [24]:
#Adding a column with values of color for every station
rm19['color'] = rm19.apply(lambda row: 'lightgray', axis=1)

#printing to check the structure of the dataframe
rm19.head(1)

,Station Name,lat,long,color
0,Orleans St & Elm St (*),41.902924,-87.637715,lightgray


# Location Marker map for interactivity purpose (Station Name)

In [25]:
#merging both the added20 and removed19 dataframes
merged = pd.concat([rm19, st20], axis =0)

In [26]:
#setting marker as the color mentioned in the color column of the dataframe
marker = merged.loc[merged['color']!='']

map = folium.Map(location=[marker.lat.mean(), marker.long.mean()], zoom_start=12,
                     width=980,height=590, control_scale=True, tiles='OpenStreetMap')
marker.apply(lambda row:folium.Marker(location=[row["lat"], row["long"]],
        popup=row['Station Name'],
          icon=folium.Icon(color=row['color'])
            ).add_to(map), axis=1)

map

# Circle marker map 2020
## Saving the map for converting into gif

In [27]:
#create a map
m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='cartodb positron')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2020')   
m.get_root().html.add_child(folium.Element(title_html))

#creates a CircleMarker and adds it to the map
def plotDot(point):
    folium.CircleMarker(location=[point.lat, point.long],             
                        color= "green",
                        radius=3,
                        weight=2
                       ).add_to(m)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
st20.apply(plotDot, axis = 1)

m

In [28]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [29]:
m.save('Gif_circlemarker/2020.html')

In [30]:
with fileinput.FileInput('Gif_circlemarker/2020.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [31]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_circlemarker')

fn='2020.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2020.png')
browser.quit()

In [32]:
image = Image.open('ss2020.png')
box = (0, 0, 1100, 700)
cropped_image = image.crop(box)
cropped_image.save('croppedss2020.png')

# Heatmap 2020
## Saving the map for converting into gif

In [33]:
numlist = st20.reset_index()[['lat', 'long']].values.tolist()


m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='OpenStreetMap')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2020')   
m.get_root().html.add_child(folium.Element(title_html))


HeatMap(numlist).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

m

In [34]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [35]:
m.save('Gif_heatmap/2020.html')

In [36]:
with fileinput.FileInput('Gif_heatmap/2020.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [37]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_heatmap')

fn='2020.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2020.png')
browser.quit()

In [39]:
image = Image.open('ss2020.png')
box = (0, 0, 1250, 780)
cropped_image = image.crop(box)
cropped_image.save('croppedss2020.png')